In [530]:
import os
import csv
import requests
import zipfile
import io
import pandas as pd
from datetime import datetime

In [531]:
from etl import etl_alpha
from utils import sql_manager
from utils import utils

In [532]:
key = os.environ.get('ALPHAVANTAGE_API_KEY')

### SQL Manager

In [559]:
sql = sql_manager.ManagerSQL()

In [560]:
query = 'select * from assets_tiingo limit 100'
sql.select_query(query)

,ticker,exchange,asset_type,price_currency,start_date,end_date
0,000001,SHE,Stock,CNY,2007-01-04,2022-09-28
1,000002,SHE,Stock,CNY,2007-01-04,2022-09-28
2,000004,SHE,Stock,CNY,2007-08-31,2022-09-28
3,000005,SHE,Stock,CNY,2007-08-31,2022-09-28
4,000006,SHE,Stock,CNY,2007-01-04,2022-09-28
...,...,...,...,...,...,...
95,000425,SHE,Stock,CNY,2007-01-04,2022-09-28
96,000426,SHE,Stock,CNY,2007-08-30,2022-09-28
97,000428,SHE,Stock,CNY,2007-08-30,2022-09-28
98,000429,SHE,Stock,CNY,2007-01-04,2022-09-28


### Update available assets

In [535]:
# if changes were made to etl_alpha
import importlib
importlib.reload(etl_alpha)

<module 'etl.etl_alpha' from '/Users/nace/dev/esg-portfolio/app/etl/etl_alpha.py'>

In [11]:
alpha = etl_alpha.AlphaScraper()

In [12]:
data = alpha.download_all_listings(datetime.now())
data

,symbol,name,exchange,asset_type,ipo_date,delisting_date,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,None,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,None,Active
2,AAA,AXS FIRST PRIORITY CLO BOND ETF,NYSE ARCA,ETF,2020-09-09,None,Active
3,AAAU,Goldman Sachs Physical Gold ETF,BATS,ETF,2018-08-15,None,Active
4,AAC,Ares Acquisition Corporation - Class A,NYSE,Stock,2021-03-25,None,Active
...,...,...,...,...,...,...,...
18435,ZSML,ETFS ZACKS EARNINGS SMALLCAP U.S. INDEX FUND,NYSE ARCA,ETF,2015-01-21,2017-09-14,Delisted
18436,ZSPH,ZS Pharma Inc,NASDAQ,Stock,2014-06-18,2015-12-16,Delisted
18437,ZTM,Agritech Worldwide Inc,NYSE MKT,Stock,2008-09-15,2020-08-12,Delisted
18438,ZU,Zulily Inc,NASDAQ,Stock,2013-11-15,2015-09-30,Delisted


In [13]:
data_clean = alpha.clean_listings(data)
data_clean

,symbol,name,asset_type,ipo_date,delisting_date
0,A,Agilent Technologies Inc,Stock,1999-11-18,None
1,AA,Alcoa Corp,Stock,2016-10-18,None
2,AAA,AXS FIRST PRIORITY CLO BOND ETF,ETF,2020-09-09,None
3,AAAU,Goldman Sachs Physical Gold ETF,ETF,2018-08-15,None
4,AAC,Ares Acquisition Corporation - Class A,Stock,2021-03-25,None
...,...,...,...,...,...
18417,PFH,Prudential Financial Inc,Stock,1999-04-05,None
18418,PLL,Piedmont Lithium Inc,Stock,1991-08-21,None
18419,RILYG,B. Riley Financial Inc,Stock,2017-12-14,None
18420,RILYZ,B. Riley Financial Inc,Stock,2017-06-01,None


In [14]:
assets = alpha.get_assets_table(data_clean)
assets

,pid,symbol,name,asset_type,ipo_date,delisting_date
0,0,A,Agilent Technologies Inc,Stock,1999-11-18,None
1,1,AA,Alcoa Corp,Stock,2016-10-18,None
2,2,AA-W,Alcoa Corporation When Issued,Stock,2016-10-18,2016-11-08
3,3,AAAP,Advanced Accelerator Applications SA,Stock,2015-11-11,2018-02-20
4,4,AABA,Altaba Inc,Stock,1996-04-12,2019-11-06
...,...,...,...,...,...,...
13753,13753,ZYME,Zymeworks Inc,Stock,2017-04-28,None
13754,13754,ZYNE,Zynerba Pharmaceuticals Inc,Stock,2015-08-05,None
13755,13755,ZYXI,Zynex Inc,Stock,2012-07-23,None
13756,13756,ZZK,,Stock,2020-07-22,None


In [15]:
#alpha.refresh_listings()

#### Daily prices

In [536]:
# if changes were made to etl_alpha
import importlib
importlib.reload(etl_alpha)

<module 'etl.etl_alpha' from '/Users/nace/dev/esg-portfolio/app/etl/etl_alpha.py'>

In [537]:
alpha = etl_alpha.AlphaScraper()

In [538]:
symbol = 'AMZN'

In [539]:
api_prices = alpha.get_adjusted_prices(symbol, size='compact')

In [540]:
api_prices.head()

,symbol,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,AMZN,2022-10-11,112.71,115.48,110.39,112.21,112.21,56432233.0,0.0,1.0
1,AMZN,2022-10-10,115.10,116.25,112.43,113.67,113.67,42339675.0,0.0,1.0
2,AMZN,2022-10-07,118.00,118.17,113.88,114.56,114.56,54677980.0,0.0,1.0
3,AMZN,2022-10-06,120.77,121.53,119.50,120.30,120.30,42253755.0,0.0,1.0
4,AMZN,2022-10-05,118.58,121.75,117.69,120.95,120.95,48217535.0,0.0,1.0


In [541]:
api_prices['lud'] = datetime.now()

In [412]:
api_prices

,symbol,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,lud
0,AMZN,2022-10-11,112.71,115.4800,110.3900,112.21,112.210,55932239.0,0.0,1.0,2022-10-11 18:34:17.233412
1,AMZN,2022-10-10,115.10,116.2500,112.4300,113.67,113.670,42339675.0,0.0,1.0,2022-10-11 18:34:17.233412
2,AMZN,2022-10-07,118.00,118.1700,113.8800,114.56,114.560,54677980.0,0.0,1.0,2022-10-11 18:34:17.233412
3,AMZN,2022-10-06,120.77,121.5300,119.5000,120.30,120.300,42253755.0,0.0,1.0,2022-10-11 18:34:17.233412
4,AMZN,2022-10-05,118.58,121.7500,117.6900,120.95,120.950,48217535.0,0.0,1.0,2022-10-11 18:34:17.233412
...,...,...,...,...,...,...,...,...,...,...,...
95,AMZN,2022-05-25,2073.11,2163.4999,2073.0000,2135.50,106.775,4656005.0,0.0,1.0,2022-10-11 18:34:17.233412
96,AMZN,2022-05-24,2080.50,2108.0000,2025.2000,2082.00,104.100,5146734.0,0.0,1.0,2022-10-11 18:34:17.233412
97,AMZN,2022-05-23,2169.22,2176.3750,2079.0000,2151.14,107.557,5389868.0,0.0,1.0,2022-10-11 18:34:17.233412
98,AMZN,2022-05-20,2191.37,2197.9599,2100.1868,2151.82,107.591,4975032.0,0.0,1.0,2022-10-11 18:34:17.233412


In [413]:
query = f"select * from prices_alpha where symbol = '{symbol}'"
db_prices = sql.select_query(query)

In [414]:
db_prices_late = db_prices.loc[(db_prices.date > datetime(2022, 10, 1).date()) & 
                               (db_prices.date < datetime(2022, 10, 7).date())]
db_prices_early = db_prices.loc[db_prices.date < datetime(2022, 6, 1).date()]

In [415]:
db_prices_mid_miss = db_prices_late.append(db_prices_early)

In [416]:
db_prices

,symbol,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,lud
0,AMZN,2022-10-10,115.10,116.25,112.43,113.67,113.67,42339675,0.0,1.0,2022-10-10 20:45:16.235821
1,AMZN,2022-10-07,118.00,118.17,113.88,114.56,114.56,54677980,0.0,1.0,2022-10-10 20:45:16.235821
2,AMZN,2022-10-06,120.77,121.53,119.50,120.30,120.30,42253755,0.0,1.0,2022-10-10 20:45:16.235821
3,AMZN,2022-10-05,118.58,121.75,117.69,120.95,120.95,48217535,0.0,1.0,2022-10-10 20:45:16.235821
4,AMZN,2022-10-04,119.89,123.00,119.79,121.09,121.09,62812564,0.0,1.0,2022-10-10 20:45:16.235821
...,...,...,...,...,...,...,...,...,...,...,...
95,AMZN,2022-05-24,2080.50,2108.00,2025.20,2082.00,104.10,5146734,0.0,1.0,2022-10-10 20:45:16.235821
96,AMZN,2022-05-23,2169.22,2176.38,2079.00,2151.14,107.56,5389868,0.0,1.0,2022-10-10 20:45:16.235821
97,AMZN,2022-05-20,2191.37,2197.96,2100.19,2151.82,107.59,4975032,0.0,1.0,2022-10-10 20:45:16.235821
98,AMZN,2022-05-19,2125.61,2200.68,2123.86,2146.38,107.32,4407127,0.0,1.0,2022-10-10 20:45:16.235821


In [417]:
db_prices_late

,symbol,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,lud
2,AMZN,2022-10-06,120.77,121.53,119.50,120.30,120.30,42253755,0.0,1.0,2022-10-10 20:45:16.235821
3,AMZN,2022-10-05,118.58,121.75,117.69,120.95,120.95,48217535,0.0,1.0,2022-10-10 20:45:16.235821
4,AMZN,2022-10-04,119.89,123.00,119.79,121.09,121.09,62812564,0.0,1.0,2022-10-10 20:45:16.235821
5,AMZN,2022-10-03,113.58,116.91,112.45,115.88,115.88,50941854,0.0,1.0,2022-10-10 20:45:16.235821


In [418]:
db_prices_early

,symbol,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,lud
91,AMZN,2022-05-31,2325.60,2439.89,2313.50,2404.19,120.21,7231708,0.0,1.0,2022-10-10 20:45:16.235821
92,AMZN,2022-05-27,2271.00,2303.74,2252.56,2302.93,115.15,4683008,0.0,1.0,2022-10-10 20:45:16.235821
93,AMZN,2022-05-26,2159.40,2253.31,2149.07,2221.55,111.08,4650130,0.0,1.0,2022-10-10 20:45:16.235821
94,AMZN,2022-05-25,2073.11,2163.50,2073.00,2135.50,106.78,4656005,0.0,1.0,2022-10-10 20:45:16.235821
95,AMZN,2022-05-24,2080.50,2108.00,2025.20,2082.00,104.10,5146734,0.0,1.0,2022-10-10 20:45:16.235821
96,AMZN,2022-05-23,2169.22,2176.38,2079.00,2151.14,107.56,5389868,0.0,1.0,2022-10-10 20:45:16.235821
97,AMZN,2022-05-20,2191.37,2197.96,2100.19,2151.82,107.59,4975032,0.0,1.0,2022-10-10 20:45:16.235821
98,AMZN,2022-05-19,2125.61,2200.68,2123.86,2146.38,107.32,4407127,0.0,1.0,2022-10-10 20:45:16.235821
99,AMZN,2022-05-18,2228.79,2257.06,2124.98,2142.25,107.11,5418959,0.0,1.0,2022-10-10 20:45:16.235821


In [419]:
db_prices_mid_miss

,symbol,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,lud
2,AMZN,2022-10-06,120.77,121.53,119.50,120.30,120.30,42253755,0.0,1.0,2022-10-10 20:45:16.235821
3,AMZN,2022-10-05,118.58,121.75,117.69,120.95,120.95,48217535,0.0,1.0,2022-10-10 20:45:16.235821
4,AMZN,2022-10-04,119.89,123.00,119.79,121.09,121.09,62812564,0.0,1.0,2022-10-10 20:45:16.235821
5,AMZN,2022-10-03,113.58,116.91,112.45,115.88,115.88,50941854,0.0,1.0,2022-10-10 20:45:16.235821
91,AMZN,2022-05-31,2325.60,2439.89,2313.50,2404.19,120.21,7231708,0.0,1.0,2022-10-10 20:45:16.235821
92,AMZN,2022-05-27,2271.00,2303.74,2252.56,2302.93,115.15,4683008,0.0,1.0,2022-10-10 20:45:16.235821
93,AMZN,2022-05-26,2159.40,2253.31,2149.07,2221.55,111.08,4650130,0.0,1.0,2022-10-10 20:45:16.235821
94,AMZN,2022-05-25,2073.11,2163.50,2073.00,2135.50,106.78,4656005,0.0,1.0,2022-10-10 20:45:16.235821
95,AMZN,2022-05-24,2080.50,2108.00,2025.20,2082.00,104.10,5146734,0.0,1.0,2022-10-10 20:45:16.235821
96,AMZN,2022-05-23,2169.22,2176.38,2079.00,2151.14,107.56,5389868,0.0,1.0,2022-10-10 20:45:16.235821


In [500]:
def find_date_to_upload_from(api_prices, db_prices):
    """
        Returns date from which api_prices should be uploaded (inclusive)
        It is also the date up to which db dates are valid and should be kept
        If output is None, no filter should be applied over api_prices
        and all registers of the symbol should be deleted from the prices table
    """

    api_dates_df = api_prices[['date', 'lud']].rename(columns={'lud': 'lud_api'})
    db_dates_df = db_prices[['date', 'lud']].rename(columns={'lud': 'lud_db'})
    dates_df = api_dates_df.merge(
        db_dates_df,
        how='outer',
        left_on='date',
        right_on='date'
    )
    db_dates_missing = dates_df[dates_df.lud_db.isnull()].date
    db_dates_valid = db_prices.loc[db_prices.date < db_dates_missing.min()]
    if db_dates_valid.shape[0] > 0:
        return db_dates_valid.date.max()
    return None

In [432]:
find_date_to_upload_from(api_prices, db_prices)

0    2022-10-11
Name: date, dtype: object


datetime.date(2022, 10, 10)

In [440]:
find_date_to_upload_from(api_prices, db_prices_early)

missing 0     2022-10-11
1     2022-10-10
2     2022-10-07
3     2022-10-06
4     2022-10-05
         ...    
87    2022-06-07
88    2022-06-06
89    2022-06-03
90    2022-06-02
91    2022-06-01
Name: date, Length: 92, dtype: object


datetime.date(2022, 5, 31)

In [443]:
upload_date = find_date_to_upload_from(api_prices, db_prices_late)
upload_date

missing 0     2022-10-11
1     2022-10-10
2     2022-10-07
7     2022-09-30
8     2022-09-29
         ...    
95    2022-05-25
96    2022-05-24
97    2022-05-23
98    2022-05-20
99    2022-05-19
Name: date, Length: 96, dtype: object


In [445]:
upload_date = find_date_to_upload_from(api_prices, db_prices_mid_miss)
upload_date

missing 0     2022-10-11
1     2022-10-10
2     2022-10-07
7     2022-09-30
8     2022-09-29
         ...    
87    2022-06-07
88    2022-06-06
89    2022-06-03
90    2022-06-02
91    2022-06-01
Name: date, Length: 88, dtype: object


datetime.date(2022, 5, 31)

In [433]:
api_prices

,symbol,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,lud
0,AMZN,2022-10-11,112.71,115.4800,110.3900,112.21,112.210,55932239.0,0.0,1.0,2022-10-11 18:34:17.233412
1,AMZN,2022-10-10,115.10,116.2500,112.4300,113.67,113.670,42339675.0,0.0,1.0,2022-10-11 18:34:17.233412
2,AMZN,2022-10-07,118.00,118.1700,113.8800,114.56,114.560,54677980.0,0.0,1.0,2022-10-11 18:34:17.233412
3,AMZN,2022-10-06,120.77,121.5300,119.5000,120.30,120.300,42253755.0,0.0,1.0,2022-10-11 18:34:17.233412
4,AMZN,2022-10-05,118.58,121.7500,117.6900,120.95,120.950,48217535.0,0.0,1.0,2022-10-11 18:34:17.233412
...,...,...,...,...,...,...,...,...,...,...,...
95,AMZN,2022-05-25,2073.11,2163.4999,2073.0000,2135.50,106.775,4656005.0,0.0,1.0,2022-10-11 18:34:17.233412
96,AMZN,2022-05-24,2080.50,2108.0000,2025.2000,2082.00,104.100,5146734.0,0.0,1.0,2022-10-11 18:34:17.233412
97,AMZN,2022-05-23,2169.22,2176.3750,2079.0000,2151.14,107.557,5389868.0,0.0,1.0,2022-10-11 18:34:17.233412
98,AMZN,2022-05-20,2191.37,2197.9599,2100.1868,2151.82,107.591,4975032.0,0.0,1.0,2022-10-11 18:34:17.233412


In [437]:
db_prices_early

,symbol,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,lud
91,AMZN,2022-05-31,2325.60,2439.89,2313.50,2404.19,120.21,7231708,0.0,1.0,2022-10-10 20:45:16.235821
92,AMZN,2022-05-27,2271.00,2303.74,2252.56,2302.93,115.15,4683008,0.0,1.0,2022-10-10 20:45:16.235821
93,AMZN,2022-05-26,2159.40,2253.31,2149.07,2221.55,111.08,4650130,0.0,1.0,2022-10-10 20:45:16.235821
94,AMZN,2022-05-25,2073.11,2163.50,2073.00,2135.50,106.78,4656005,0.0,1.0,2022-10-10 20:45:16.235821
95,AMZN,2022-05-24,2080.50,2108.00,2025.20,2082.00,104.10,5146734,0.0,1.0,2022-10-10 20:45:16.235821
96,AMZN,2022-05-23,2169.22,2176.38,2079.00,2151.14,107.56,5389868,0.0,1.0,2022-10-10 20:45:16.235821
97,AMZN,2022-05-20,2191.37,2197.96,2100.19,2151.82,107.59,4975032,0.0,1.0,2022-10-10 20:45:16.235821
98,AMZN,2022-05-19,2125.61,2200.68,2123.86,2146.38,107.32,4407127,0.0,1.0,2022-10-10 20:45:16.235821
99,AMZN,2022-05-18,2228.79,2257.06,2124.98,2142.25,107.11,5418959,0.0,1.0,2022-10-10 20:45:16.235821


In [381]:
db_prices_late

,symbol,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,lud
2,AMZN,2022-10-06,120.77,121.53,119.50,120.30,120.30,42253755,0.0,1.0,2022-10-10 20:45:16.235821
3,AMZN,2022-10-05,118.58,121.75,117.69,120.95,120.95,48217535,0.0,1.0,2022-10-10 20:45:16.235821
4,AMZN,2022-10-04,119.89,123.00,119.79,121.09,121.09,62812564,0.0,1.0,2022-10-10 20:45:16.235821
5,AMZN,2022-10-03,113.58,116.91,112.45,115.88,115.88,50941854,0.0,1.0,2022-10-10 20:45:16.235821


In [383]:
find_date_to_upload_from(api_prices, db_prices_late)

0     2022-10-11
1     2022-10-10
2     2022-10-07
7     2022-09-30
8     2022-09-29
         ...    
95    2022-05-25
96    2022-05-24
97    2022-05-23
98    2022-05-20
99    2022-05-19
Name: date, Length: 96, dtype: object


NaT

In [447]:
upload_date.strftime('%Y-%m-%d')

'2022-05-31'

In [450]:
table_prices = 'prices_alpha'

In [456]:
query = f"""
select * 
from {table_prices}
where
    symbol = '{symbol}' and 
    date > '{upload_date.strftime('%Y-%m-%d')}'
"""
query

"\nselect * \nfrom prices_alpha\nwhere\n    symbol = 'AMZN' and \n    date > '2022-05-31'\n"

In [457]:
sql.select_query(query)

,symbol,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,lud
0,AMZN,2022-10-10,115.10,116.25,112.43,113.67,113.67,42339675,0.0,1.0,2022-10-10 20:45:16.235821
1,AMZN,2022-10-07,118.00,118.17,113.88,114.56,114.56,54677980,0.0,1.0,2022-10-10 20:45:16.235821
2,AMZN,2022-10-06,120.77,121.53,119.50,120.30,120.30,42253755,0.0,1.0,2022-10-10 20:45:16.235821
3,AMZN,2022-10-05,118.58,121.75,117.69,120.95,120.95,48217535,0.0,1.0,2022-10-10 20:45:16.235821
4,AMZN,2022-10-04,119.89,123.00,119.79,121.09,121.09,62812564,0.0,1.0,2022-10-10 20:45:16.235821
...,...,...,...,...,...,...,...,...,...,...,...
86,AMZN,2022-06-07,122.01,124.10,120.63,123.00,123.00,85156712,0.0,1.0,2022-10-10 20:45:16.235821
87,AMZN,2022-06-06,125.25,128.99,123.81,124.79,124.79,134271125,0.0,20.0,2022-10-10 20:45:16.235821
88,AMZN,2022-06-03,2484.00,2488.00,2420.93,2447.00,122.35,4880166,0.0,1.0,2022-10-10 20:45:16.235821
89,AMZN,2022-06-02,2433.68,2512.20,2400.90,2510.22,125.51,5028034,0.0,1.0,2022-10-10 20:45:16.235821


In [542]:
def update_prices(api_prices_input, table_prices='prices_alpha'):

    api_prices = api_prices_input.copy()

    # Symbol
    symbols = api_prices.symbol.unique()
    assert len(symbols) == 1, f"Only one symbol per update. symbols={symbols}"
    symbol = symbols[0]

    # Add or update lud column
    api_prices['lud'] = datetime.now()

    # Get last valid date of symbol in database (inclusive)
    # Which is also the same from which api_prices should be uploaded (inclusive)
    date_upload = find_date_to_upload_from(api_prices, db_prices)
    print(date_upload)

    # Filter data from API to start from last date of symbol in db
    if date_upload is None:
        # Clean
        query = f"delete from {table_prices} where symbol = '{symbol}'"
        print(query)
        sql.query(query)
    else:
        # Clean
        query = f"""
        delete from {table_prices}
        where
            symbol = '{symbol}' and 
            date > '{date_upload.strftime('%Y-%m-%d')}'
        """
        print(query)
        sql.query(query)
        
        # Filter api_prices
        api_prices = api_prices.loc[api_prices['date'] >= date_upload]

    # Upload to database
    print(api_prices)
    if api_prices.shape[0] > 1:
        print(f'Uploading {api_prices.shape[0]} rows for {symbol}')
        sql.upload_df_chunks(table_prices, api_prices)
    else:
        print("Database already up to date.")

In [544]:
size = 'compact'

In [545]:
prices_amzn = alpha.get_adjusted_prices(symbol='AMZN', size=size)

In [528]:
prices_msft = alpha.get_adjusted_prices(symbol='MSFT', size=size)

In [570]:
sql.select_query("select symbol, min(date) min_date, max(date) max_date from prices_alpha group by symbol")

,symbol,min_date,max_date
0,AMZN,1999-11-01,2022-10-12
1,AAPL,2022-05-20,2022-10-12
2,MSFT,2022-05-18,2022-10-12


In [571]:
sql.select_query("""select symbol, date, lud, close, adjusted_close from prices_alpha where symbol = 'AMZN' 
                 order by date desc, lud desc""").head(10)

,symbol,date,lud,close,adjusted_close
0,AMZN,2022-10-12,2022-10-12 23:10:25.021533,112.90,112.90
1,AMZN,2022-10-11,2022-10-12 23:10:25.021533,112.21,112.21
2,AMZN,2022-10-10,2022-10-12 23:10:25.021533,113.67,113.67
3,AMZN,2022-10-07,2022-10-12 23:10:25.021533,114.56,114.56
4,AMZN,2022-10-06,2022-10-12 23:10:25.021533,120.30,120.30
5,AMZN,2022-10-05,2022-10-12 23:10:25.021533,120.95,120.95
6,AMZN,2022-10-04,2022-10-12 23:10:25.021533,121.09,121.09
7,AMZN,2022-10-03,2022-10-12 23:10:25.021533,115.88,115.88
8,AMZN,2022-09-30,2022-10-12 23:10:25.021533,113.00,113.00
9,AMZN,2022-09-29,2022-10-12 23:10:25.021533,114.80,114.80


In [569]:
sql.query("delete from prices_alpha where symbol = 'AMZN'")

In [572]:
sql.query("delete from prices_alpha where symbol = 'AMZN' and date = '2022-10-12'")

In [547]:
prices_amzn.shape

(100, 10)

In [548]:
prices_msft.shape

(100, 10)

In [551]:
update_prices(prices_amzn)

2022-10-10

        delete from prices_alpha
        where
            symbol = 'AMZN' and 
            date > '2022-10-10'
        
  symbol        date    open    high     low   close  adjusted_close  \
0   AMZN  2022-10-11  112.71  115.48  110.39  112.21          112.21   
1   AMZN  2022-10-10  115.10  116.25  112.43  113.67          113.67   

       volume  dividend_amount  split_coefficient                        lud  
0  56432233.0              0.0                1.0 2022-10-11 22:00:15.065374  
1  42339675.0              0.0                1.0 2022-10-11 22:00:15.065374  
Uploading 2 rows for AMZN


In [513]:
update_prices(prices_msft)

2022-10-10

        delete from prices_alpha
        where
            symbol = 'MSFT' and 
            date > '2022-10-10'
        
  symbol        date    open     high     low   close  adjusted_close  \
0   MSFT  2022-10-11  227.62  229.060  224.11  225.41          225.41   
1   MSFT  2022-10-10  233.05  234.559  226.73  229.25          229.25   

       volume  dividend_amount  split_coefficient                        lud  
0  30159800.0              0.0                1.0 2022-10-11 19:07:13.475756  
1  29743608.0              0.0                1.0 2022-10-11 19:07:13.475756  
Uploading 2 rows for MSFT


In [297]:
prices_amzn.append(prices_msft)

,symbol,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,lud
0,AMZN,2022-10-10,115.10,116.25,112.43,113.67,113.670000,42339675.0,0.00,1.0,2022-10-10 22:22:09.659057
1,AMZN,2022-10-07,118.00,118.17,113.88,114.56,114.560000,54677980.0,0.00,1.0,2022-10-10 22:22:09.659057
2,AMZN,2022-10-06,120.77,121.53,119.50,120.30,120.300000,42253755.0,0.00,1.0,2022-10-10 22:22:09.659057
3,AMZN,2022-10-05,118.58,121.75,117.69,120.95,120.950000,48217535.0,0.00,1.0,2022-10-10 22:22:09.659057
4,AMZN,2022-10-04,119.89,123.00,119.79,121.09,121.090000,62812564.0,0.00,1.0,2022-10-10 22:22:09.659057
...,...,...,...,...,...,...,...,...,...,...,...
95,MSFT,2022-05-24,257.89,261.33,253.50,259.62,259.068639,29043904.0,0.00,1.0,2022-10-10 22:25:19.301635
96,MSFT,2022-05-23,255.49,261.50,253.43,260.65,260.096451,33175379.0,0.00,1.0,2022-10-10 22:25:19.301635
97,MSFT,2022-05-20,257.24,258.54,246.44,252.56,252.023632,39199279.0,0.00,1.0,2022-10-10 22:25:19.301635
98,MSFT,2022-05-19,253.90,257.67,251.88,253.14,252.602400,32692286.0,0.00,1.0,2022-10-10 22:25:19.301635


In [298]:
update_prices(prices_amzn.append(prices_msft))

AssertionError: Only one symbol per update. symbols=['AMZN' 'MSFT']

In [ ]:
# CAREFUL WITH COMPACT!!

In [279]:
symbols[0]

'AMZN'

In [118]:
data_last = data.copy()
data_last['delisting_date'] = data_last.delisting_date.fillna(datetime.now().date().strftime('%Y-%m-%d'))
data_last = data_last.sort_values(by=['delisting_date', 'symbol'], ascending=[False, True])
data_last = data_last.drop_duplicates(subset=['symbol'], keep=False)

In [119]:
data_last

,symbol,name,exchange,asset_type,ipo_date,delisting_date,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,2022-10-06,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,2022-10-06,Active
2,AAA,AXS FIRST PRIORITY CLO BOND ETF,NYSE ARCA,ETF,2020-09-09,2022-10-06,Active
3,AAAU,Goldman Sachs Physical Gold ETF,BATS,ETF,2018-08-15,2022-10-06,Active
5,AAC-U,Ares Acquisition Corporation - Units (1 Ord Sh...,NYSE,Stock,2021-02-02,2022-10-06,Active
...,...,...,...,...,...,...,...
15418,IKN,IKON OFFICE SOLUTIONS INC,NYSE,Stock,2006-12-28,2008-11-21,Delisted
14346,EFD,EFUNDS CORP,NYSE,Stock,2006-12-28,2007-11-13,Delisted
14994,GNSM,GENSYM CORP,NASDAQ,Stock,2007-07-13,2007-10-25,Delisted
13076,ARS,Aleris International Inc,NASDAQ,Stock,2006-12-19,2006-12-29,Delisted


In [19]:
col_keys = ['symbol', 'name', 'asset_type']
data_no_dup = data.drop_duplicates(subset=col_keys, keep=False)
ind_dup = [ind for ind in data.index if ind not in data_no_dup.index]
data_dup = data.iloc[ind_dup]

In [20]:
dup_sym = data.loc[data.symbol.isin(data_dup.symbol)]

In [21]:
dup_sym.sort_values(by=['symbol', 'name', 'exchange', 'asset_type'])

,symbol,name,exchange,asset_type,ipo_date,delisting_date,status
13088,ARVR,FIRST TRUST INDXX METAVERSE ETF,BATS,ETF,2016-12-09,2018-12-24,Delisted
807,ARVR,FIRST TRUST INDXX METAVERSE ETF,NASDAQ,ETF,2022-04-20,None,Active
1838,BXRX,Baudax Bio Inc,NASDAQ,Stock,2019-11-22,None,Active
13480,BXRX,Baudax Bio Inc,NASDAQ,Stock,2019-11-14,2019-11-21,Delisted
1880,CADE,Cadence Bancorporation - Class A,NYSE,Stock,1985-10-16,None,Active
13500,CADE,Cadence Bancorporation - Class A,NYSE,Stock,2017-04-13,2021-10-28,Delisted
2887,DCOM,Dime Community Bancshares Inc,NASDAQ,Stock,1999-01-12,None,Active
14066,DCOM,Dime Community Bancshares Inc,NASDAQ,Stock,1996-06-26,2021-01-29,Delisted
3120,DNB,Dun & Bradstreet Holdings Inc,NYSE,Stock,2020-07-01,None,Active
14181,DNB,Dun & Bradstreet Holdings Inc,NYSE,Stock,1983-06-10,2021-04-01,Delisted


In [70]:
symbol = 'ARVR'
data.loc[data.symbol == symbol]

,symbol,name,exchange,asset_type,ipo_date,delisting_date,status
807,ARVR,FIRST TRUST INDXX METAVERSE ETF,NASDAQ,ETF,2022-04-20,None,Active
13088,ARVR,FIRST TRUST INDXX METAVERSE ETF,BATS,ETF,2016-12-09,2018-12-24,Delisted


In [71]:
df = alpha.get_prices(symbol)
df.sort_values('date')

,symbol,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
117,ARVR,2022-04-20,29.59,29.61,29.5299,29.5299,29.5299,609,0.0000,1.0
116,ARVR,2022-04-21,29.3999,29.3999,28.7176,28.7176,28.7176,384,0.0000,1.0
115,ARVR,2022-04-22,28.3715,28.3715,28.3715,28.3715,28.3715,7,0.0000,1.0
114,ARVR,2022-04-25,28.7378,28.7378,28.7378,28.7378,28.7378,7,0.0000,1.0
113,ARVR,2022-04-26,27.7676,27.7676,27.7676,27.7676,27.7676,87,0.0000,1.0
...,...,...,...,...,...,...,...,...,...,...
4,ARVR,2022-09-30,22.4037,22.4037,22.4037,22.4037,22.4037,0,0.0000,1.0
3,ARVR,2022-10-03,23.0392,23.0392,23.0392,23.0392,23.0392,13,0.0000,1.0
2,ARVR,2022-10-04,23.922,23.922,23.922,23.922,23.922,24,0.0000,1.0
1,ARVR,2022-10-05,23.81,23.81,23.81,23.81,23.81,1,0.0000,1.0


In [72]:
url = f'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={symbol}&apikey={alpha.api_key}'

# Hit API
r = requests.get(url)
json_file = r.json()
json_file

{'bestMatches': [{'1. symbol': 'ARVR',
   '2. name': 'FIRST TRUST INDXX METAVERSE ETF ',
   '3. type': 'ETF',
   '4. region': 'United States',
   '5. marketOpen': '09:30',
   '6. marketClose': '16:00',
   '7. timezone': 'UTC-04',
   '8. currency': 'USD',
   '9. matchScore': '1.0000'}]}

In [57]:
symbol = 'ARVR.BATS'
df = alpha.get_prices(symbol)
df.sort_values('date')

Download failed for ARVR.BATS. 
url: https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=ARVR.BATS&outputsize=full&apikey=9OQ55ETVC6OQ4YSF
Json file did not have 'Time Series (Daily)' as key. File content is:
 {'Error Message': 'Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY_ADJUSTED.'}


AttributeError: 'NoneType' object has no attribute 'sort_values'

In [73]:
symbol = 'BXRX'
data.loc[data.symbol == symbol]

,symbol,name,exchange,asset_type,ipo_date,delisting_date,status
1838,BXRX,Baudax Bio Inc,NASDAQ,Stock,2019-11-22,None,Active
13480,BXRX,Baudax Bio Inc,NASDAQ,Stock,2019-11-14,2019-11-21,Delisted


In [74]:
url = f'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={symbol}&apikey={alpha.api_key}'

# Hit API
r = requests.get(url)
json_file = r.json()
json_file

{'bestMatches': [{'1. symbol': 'BXRX',
   '2. name': 'Baudax Bio Inc',
   '3. type': 'Equity',
   '4. region': 'United States',
   '5. marketOpen': '09:30',
   '6. marketClose': '16:00',
   '7. timezone': 'UTC-04',
   '8. currency': 'USD',
   '9. matchScore': '1.0000'}]}

In [75]:
df = alpha.get_prices(symbol)
df.sort_values('date')

,symbol,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
722,BXRX,2019-11-22,4.12,4.84,3.6,4.48,150.552988048,583718,0.0000,1.0
721,BXRX,2019-11-25,3.58,4.6,3.57,4.56,153.241434263,149094,0.0000,1.0
720,BXRX,2019-11-26,4.76,8.41,4.605,6.86,230.534262948,694465,0.0000,1.0
719,BXRX,2019-11-27,7.11,7.72,5.2,5.6,188.19123506,667053,0.0000,1.0
718,BXRX,2019-11-29,5.54,5.7,5.2,5.37,180.461952191,142314,0.0000,1.0
...,...,...,...,...,...,...,...,...,...,...
4,BXRX,2022-09-30,0.2385,0.25,0.22,0.2459,0.2459,667994,0.0000,1.0
3,BXRX,2022-10-03,0.2425,0.2439,0.226,0.2397,0.2397,462151,0.0000,1.0
2,BXRX,2022-10-04,0.2325,0.255,0.2311,0.255,0.255,1246289,0.0000,1.0
1,BXRX,2022-10-05,0.265,0.306,0.2412,0.2666,0.2666,2325574,0.0000,1.0


In [87]:
symbol = 'ZX'
data.loc[data.symbol == symbol]

,symbol,name,exchange,asset_type,ipo_date,delisting_date,status
18439,ZX,China Zenix Auto International Ltd,NYSE,Stock,2011-05-16,2018-06-14,Delisted


In [88]:
url = f'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={symbol}&apikey={alpha.api_key}'

# Hit API
r = requests.get(url)
json_file = r.json()
json_file

{'bestMatches': [{'1. symbol': 'ZXAIY',
   '2. name': 'China Zenix Auto International Ltd',
   '3. type': 'Equity',
   '4. region': 'United States',
   '5. marketOpen': '09:30',
   '6. marketClose': '16:00',
   '7. timezone': 'UTC-04',
   '8. currency': 'USD',
   '9. matchScore': '0.5714'},
  {'1. symbol': 'ZXZZT',
   '2. name': 'NASDAQ TEST STOCK',
   '3. type': 'Equity',
   '4. region': 'United States',
   '5. marketOpen': '09:30',
   '6. marketClose': '16:00',
   '7. timezone': 'UTC-04',
   '8. currency': 'USD',
   '9. matchScore': '0.5714'},
  {'1. symbol': 'ZXM.TRT',
   '2. name': 'First Asset Morningstar International Momentum\xa0CAD Hedged Common',
   '3. type': 'Equity',
   '4. region': 'Toronto',
   '5. marketOpen': '09:30',
   '6. marketClose': '16:00',
   '7. timezone': 'UTC-05',
   '8. currency': 'CAD',
   '9. matchScore': '0.5000'},
  {'1. symbol': 'ZXYZ-A',
   '2. name': 'NASDAQ SYMBOLOGY TEST',
   '3. type': 'Equity',
   '4. region': 'United States',
   '5. marketOpen': 

In [89]:
df = alpha.get_prices(symbol)
df.sort_values('date')

,symbol,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
1782,ZX,2011-05-16,6.05,6.64,6.02,6.3,6.20104712042,515200,0.0000,1.0
1781,ZX,2011-05-17,6.25,6.25,6.04,6.04,5.94513089005,321200,0.0000,1.0
1780,ZX,2011-05-18,6.12,6.15,6.08,6.08,5.9845026178,162300,0.0000,1.0
1779,ZX,2011-05-19,6.14,6.14,6.0,6.05,5.95497382199,787100,0.0000,1.0
1778,ZX,2011-05-20,6.1,6.1,5.99,6.0095,5.91510994764,174300,0.0000,1.0
...,...,...,...,...,...,...,...,...,...,...
4,ZX,2018-06-08,1.32,1.35,1.32,1.3306,1.3306,8482,0.0000,1.0
3,ZX,2018-06-11,1.34,1.35,1.3,1.35,1.35,17278,0.0000,1.0
2,ZX,2018-06-12,1.35,1.37,1.35,1.35,1.35,4017,0.0000,1.0
1,ZX,2018-06-13,1.35,1.38,1.3287,1.38,1.38,15459,0.0000,1.0


In [67]:
symbol = 'ARVR'
data.loc[data.symbol == symbol]

,symbol,name,exchange,asset_type,ipo_date,delisting_date,status
807,ARVR,FIRST TRUST INDXX METAVERSE ETF,NASDAQ,ETF,2022-04-20,None,Active
13088,ARVR,FIRST TRUST INDXX METAVERSE ETF,BATS,ETF,2016-12-09,2018-12-24,Delisted


In [68]:
url = f'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={symbol}&apikey={alpha.api_key}'

# Hit API
r = requests.get(url)
json_file = r.json()
json_file

In [43]:
url = f'{etl_alpha.URL_BASE}TIME_SERIES_DAILY_ADJUSTED&symbol=ARVR.BATS&apikey={alpha.api_key}'

# Hit API
r = requests.get(url)
prices_json = r.json()

In [44]:
prices_json

{'Error Message': 'Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY_ADJUSTED.'}

In [455]:
df.head()

,symbol,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,IBM,2022-10-06,124.88,125.3,121.77,122.23,122.23,5068773,0.0000,1.0
1,IBM,2022-10-05,124.71,126.46,124.23,125.74,125.74,3212872,0.0000,1.0
2,IBM,2022-10-04,122.8,125.65,122.52,125.5,125.5,4566055,0.0000,1.0
3,IBM,2022-10-03,120.16,122.21,119.6,121.51,121.51,4261729,0.0000,1.0
4,IBM,2022-09-30,121.66,122.43,118.61,118.81,118.81,5346108,0.0000,1.0


In [ ]:
# balance sheet
url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol=IBM&apikey=demo'
r = requests.get(url)
data = r.json()
data

In [ ]:
# income statement
url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol=IBM&apikey=demo'
r = requests.get(url)
data = r.json()
data

In [ ]:
# income statement
url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol=IBM&apikey=demo'
r = requests.get(url)
data = r.json()
data

In [ ]:
#data

In [ ]:
https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=demo